In [ ]:
!pip install crewai

In [ ]:
!pip install 'crewai[tools]'

In [ ]:
!pip install langchain_groq

In [ ]:
import os
from getpass import getpass

A content creation agent’s job could entail brainstorming content ideas, and drafting articles or blog posts.

Calling another agent for content trend insights.
Generating a list of content ideas based on those trends and target audience preferences.
Drafting articles or blog posts.
Taking instruction from a human user on edits and adjustments.

In [ ]:
SERPER_API_KEY = getpass("Your serper api key")
os.environ['SERPER_API_KEY'] = SERPER_API_KEY

GROQ_API_KEY = getpass("Your Groq api key")
os.environ['GROQ_API_KEY'] = GROQ_API_KEY

Your serper api key··········
Your Groq api key··········


In [ ]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool
from langchain_groq import ChatGroq

In [ ]:
search_tool = SerperDevTool()

In [ ]:
!ls

sample_data


In [ ]:
llm = ChatGroq(model='mixtral-8x7b-32768',groq_api_key=GROQ_API_KEY)

In [ ]:
researcher = Agent(
  role='Researcher',
  goal='Pioneer revolutionary advancements in {subject_area}',
  verbose=True,
  backstory=(
    "As a visionary researcher, your insatiable curiosity drives you"
    "to delve deep into emerging fields. With a passion for innovation"
    "and a dedication to scientific discovery, you seek to"
    "develop technologies and solutions that could transform"
    "the future."
  ),
  llm = llm,
  max_iter=5,# for good response, keep it >10
  tools=[search_tool],
  allow_delegation=True #True because, we are delegating this task to another Agent
)

writer = Agent(
  role='Writer',
  goal='Craft engaging and insightful narratives about {subject_area}',
  verbose=True,
  backstory=(
    "You are a skilled storyteller with a talent for demystifying"
    "complex innovations. Your writing illuminates the significance"
    "of new technological discoveries, connecting them with everyday"
    "lives and broader societal impacts, thereby educating"
    "and inspiring your audience."
  ),
  max_iter=5, # for good response, keep it >10
  tools=[search_tool],
  llm = llm,
  allow_delegation=False
)

In [ ]:
research_task = Task(
  description=(
    "Explore and identify the next major development in emerging technologies within {subject_area}. "
    "Evaluate the advantages and drawbacks of these technologies, and construct a comprehensive narrative."
    "Your final report should succinctly outline the key features, market potential, and possible risks associated with these emerging technologies."
  ),
  expected_output='A comprehensive report, structured into three detailed paragraphs, on the latest emerging technologies in {subject_area}.',
  tools=[search_tool],
  agent=researcher,
)

write_task = Task(
  description=(
    "Craft an engaging article on recent advancements in emerging technologies within {subject_area}. "
    "Highlight how these advancements are reshaping the industry, emphasizing their positive impacts and widespread influence."
    "The article should be clear, captivating, and optimistic, tailored for a broad audience."
  ),
  expected_output='A four-paragraph article on recent advancements in emerging technologies in {subject_area}, formatted in markdown.',
  tools=[search_tool],
  agent=writer,
  async_execution=False,
  output_file='blog.md'
)

In [ ]:
crew = Crew(
  agents=[researcher, writer],
  tasks=[research_task, write_task],
  process=Process.sequential,  # Optional: Sequential task execution is default
  cache=True,
  max_rpm= 3, #Maximum requests per minute the crew adheres to during execution.
  )

In [ ]:
result = crew.kickoff(inputs={'subject_area': "Indian Elections 2024"})



> Entering new CrewAgentExecutor chain...
I need to find out the latest emerging technologies that could be used in Indian Elections 2024. To do this, I will first use the "Search the internet" tool to gather information about the current trends and technologies in election processes.

Action: Search the internet
Action Input: {'search_query': 'emerging technologies in Indian Elections 2024'}
 


Search results: Title: Locating the Technology Agenda in India's 2024 General Election
Link: https://www.csis.org/analysis/locating-technology-agenda-indias-2024-general-election
Snippet: With the ongoing Lok Sabha general elections in India, the Chair in U.S.-India Policy Studies proposes new avenues for collaboration in the ...
---
Title: Dispatches on Tech and Democracy: India's 2024 Elections - #1
Link: https://techpolicy.press/dispatches-on-tech-and-democracy-indias-2024-elections-1
Snippet: Amber Sinha explores the role of AI and other technologies in India's 2024 elections and their i

In [ ]:
print(result)

# The Future of Indian Elections: How AI is Reshaping the Political Landscape

The Indian elections of 2024 are set to be significantly influenced by the burgeoning use of Artificial Intelligence (AI). This technology is poised to transform the nature of political campaigns and amplify voter outreach in unprecedented ways. One of the primary advantages of AI is its ability to analyze vast amounts of data, providing political parties with unparalleled insights into voter behavior and preferences. This data-driven approach allows for more strategic campaigning, including targeted messaging and increased personalization. Moreover, AI can automate various election-related tasks, thereby enhancing efficiency and reducing human error.

The integration of AI in Indian elections has the potential to yield significant positive impacts. For instance, AI-powered chatbots can help voters make informed decisions by providing relevant and accurate information about candidates, their policies, and vo